### How to write your own Energy Disaggregation algorithm

This notebook will teach you how to write your own energy disaggregation algorithm, and we will take Seq2Seq_CNN [1] as an example.  

### Steps

- Firstly, before you write your code, you should know that there are three different **mapping** in energy disaggregation algorithms, namely **Seq2Seq**(such as `DAE`, `EnerGAN` and `Seq2Seq_Attention` we implemented), **Seq2Point**(such as `Seq2Point` and `BiLSTM` we implemented) and **Seq2Subseq**(such as `SGN` we implemented).

  The math expressions behind them are,

   $f_{seq2seq}:Y_{ t-\lfloor \frac{W}{2} \rfloor: t+\lfloor \frac{W}{2} \rfloor} \rightarrow X_{ t-\lfloor \frac{W}{2} \rfloor: t+\lfloor \frac{W}{2} \rfloor} $

  $f_{seq2point}:Y_{ t-\lfloor \frac{W}{2} \rfloor: t+\lfloor \frac{W}{2} \rfloor} \rightarrow X_{ t} $

  $f_{seq2subseq}:Y_{ t-\lfloor \frac{W}{2} \rfloor: t+\lfloor \frac{W}{2} \rfloor} \rightarrow X_{ t-\lfloor \frac{w'}{2} \rfloor: t+\lfloor \frac{w'}{2} \rfloor}(w'<W) $

  Where $Y_{a:b}$ is the **mains reading** from time step a to b and $X_{a:b}$ is the **appliance reading** from time step a to b. Additionally, $t$ represents the **middle point time step** of the given sliding window, and $W$ is the **length** of the given sliding window.

  The schematic diagram of them is as follow,

[![BWMpvQ.md.png](https://s1.ax1x.com/2020/11/05/BWMpvQ.md.png)](https://imgchr.com/i/BWMpvQ)


  When disaggregating mains reading, the sliding windows slide as follow,

[![BWMCuj.md.png](https://s1.ax1x.com/2020/11/05/BWMCuj.md.png)](https://imgchr.com/i/BWMCuj)

**Different mappings require different data pipeline, so take care !**

- Secondly, you should **import packages** we will use and **fix the random seed**, you can easily do this by **copying related code** from other energy disaggregation algorithms such as BiLSTM.py

In [1]:
# Package import
from __future__ import print_function, division
from warnings import warn
from nilmtk.disaggregate import Disaggregator
import os
import pickle
import pandas as pd
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import sys
import torch
import torch.nn as nn
import torch.utils.data as tud
from torch.utils.data.dataset import TensorDataset
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import time

# Fix the random seed to ensure the reproducibility of the experiment
random_seed = 10
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Use cuda or not
USE_CUDA = torch.cuda.is_available()

C:\Users\think\Miniconda3\envs\mypytorch\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


- Thirdly, **build your own neural network** as you usually do with Pytorch and describe the **initialization method**.(Refer to **Seq2Point** to know usual **CNN initialization** method and **BiLSTM** for **RNN initialization**)

In [2]:
class seq2seqcnn_Pytorch(nn.Module):
    def __init__(self, sequence_length):
        # Refer to "ZHANG C, ZHONG M, WANG Z, et al. Sequence-to-point learning with neural networks for non-intrusive load monitoring[C].The 32nd AAAI Conference on Artificial Intelligence"
        super(seq2seqcnn_Pytorch, self).__init__()
        self.seq_length = sequence_length

        self.conv = nn.Sequential(
            nn.ConstantPad1d((4, 5), 0),
            nn.Conv1d(1, 30, 10, stride=1),
            nn.ReLU(True),
            nn.ConstantPad1d((3, 4), 0),
            nn.Conv1d(30, 30, 8, stride=1),
            nn.ReLU(True),
            nn.ConstantPad1d((2, 3), 0),
            nn.Conv1d(30, 40, 6, stride=1),
            nn.ReLU(True),
            nn.ConstantPad1d((2, 2), 0),
            nn.Conv1d(40, 50, 5, stride=1),
            nn.ReLU(True),
            nn.ConstantPad1d((2, 2), 0),
            nn.Conv1d(50, 50, 5, stride=1),
            nn.ReLU(True)
        )

        self.dense = nn.Sequential(
            nn.Linear(50 * sequence_length, 1024), 
            nn.ReLU(),
            nn.Linear(1024, sequence_length)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.dense(x.view(-1, 50 * self.seq_length))
        return x.view(-1, sequence_length)
    
def initialize(layer):
    # Xavier_uniform will be applied to conv1d and dense layer, to be sonsistent with Keras and Tensorflow
    if isinstance(layer,nn.Conv1d) or isinstance(layer, nn.Linear):    
        torch.nn.init.xavier_uniform_(layer.weight.data)
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias.data, val = 0.0)

- Then, write **network training** method. You can **copy** most of the code from other energy disaggregation algorithms, unless you train a **GAN** which requires adversarial training. Notice that **the number of appliance reading dimensions** is different with **different mapping approach** we mentioned before.

In [3]:
def train(appliance_name,model, mains, appliance, epochs, batch_size, pretrain, checkpoint_interval = None,  train_patience = 3):
    # Model configuration
    if USE_CUDA:
        model = model.cuda()
    if not pretrain:
        model.apply(initialize)
    summary(model, (1, mains.shape[1]))
    # Split the train and validation set
    train_mains,valid_mains,train_appliance,valid_appliance = train_test_split(mains, appliance, test_size=.2, random_state = random_seed)

    # Create optimizer, loss function, and dataloader
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    loss_fn = torch.nn.MSELoss(reduction = 'mean')
	# Notice that the number of appliance reading dimensions is different with different mapping approach we mentioned before
    train_dataset = TensorDataset(torch.from_numpy(train_mains).float().permute(0,2,1), torch.from_numpy(train_appliance).float().permute(0,2,1))
    train_loader = tud.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0, drop_last = True)

    valid_dataset = TensorDataset(torch.from_numpy(valid_mains).float().permute(0,2,1), torch.from_numpy(valid_appliance).float().permute(0,2,1))
    valid_loader = tud.DataLoader(valid_dataset, batch_size = batch_size, shuffle = True, num_workers = 0, drop_last = True)

    writer = SummaryWriter(comment = 'train_visual')
    patience, best_loss = 0, None

    for epoch in range(epochs):
        # Earlystopping
        if(patience == train_patience):
            print("val_loss did not improve after {} Epochs, thus Earlystopping is calling".format(train_patience))
            break   
        # Train the model
        st = time.time() 
        model.train()
            
        for i, (batch_mains, batch_appliance) in enumerate(train_loader):
            if USE_CUDA:
                batch_mains = batch_mains.cuda()
                batch_appliance = batch_appliance.cuda()
            
            batch_pred = model(batch_mains)
            loss = loss_fn(batch_pred, batch_appliance)

            model.zero_grad()    
            loss.backward()
            optimizer.step()
        ed = time.time()

        # Evaluate the model 
        model.eval()
        with torch.no_grad():
            cnt, loss_sum = 0, 0
            for i, (batch_mains, batch_appliance) in enumerate(valid_loader):
                if USE_CUDA:
                    batch_mains = batch_mains.cuda()
                    batch_appliance = batch_appliance.cuda()
            
                batch_pred = model(batch_mains)
                loss = loss_fn(batch_appliance, batch_pred)
                loss_sum += loss
                cnt += 1        

        final_loss = loss_sum / cnt
        # Save best only
        if best_loss is None or final_loss < best_loss:
            best_loss = final_loss
            patience = 0
            net_state_dict = model.state_dict()
            path_state_dict = "./"+appliance_name+"_seq2seqcnn_best_state_dict.pt"
            torch.save(net_state_dict, path_state_dict)
        else:
            patience = patience + 1 

        print("Epoch: {}, Valid_Loss: {}, Time consumption: {}s.".format(epoch, final_loss, ed - st))
        # For the visualization of training process
        for name,param in model.named_parameters():
            writer.add_histogram(name + '_grad', param.grad, epoch)
            writer.add_histogram(name + '_data', param, epoch)
        writer.add_scalars("MSELoss", {"Valid":final_loss}, epoch)

        # Save checkpoint
        if (checkpoint_interval != None) and ((epoch + 1) % checkpoint_interval == 0):
            checkpoint = {"model_state_dict": model.state_dict(),
                            "optimizer_state_dict": optimizer.state_dict(),
                            "epoch": epoch}
            path_checkpoint = "./"+appliance_name+"_seq2seqcnn_checkpoint_{}_epoch.pt".format(epoch)
            torch.save(checkpoint, path_checkpoint)

- Then, write **network testing** method. You can **copy** the code from other energy disaggregation algorithms since they **share the same testing method**.

In [4]:
def test(model, test_mains, batch_size = 512):
    # Model test
    st = time.time()
    model.eval()
    # Create test dataset and dataloader
    batch_size = test_mains.shape[0] if batch_size > test_mains.shape[0] else batch_size
    test_dataset = TensorDataset(torch.from_numpy(test_mains).float().permute(0,2,1))
    test_loader = tud.DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)
    with torch.no_grad():
        for i, batch_mains in enumerate(test_loader):
            batch_pred = model(batch_mains[0])
            if i == 0:
                res = batch_pred
            else:
                res = torch.cat((res, batch_pred), dim = 0)
    ed = time.time()
    print("Inference Time consumption: {}s.".format(ed - st))
    return res.numpy()

- Then, complement the Seq2SeqCNN class, there are several functions you should implement. 

  - `partial_fit`: **Reshape** the training data and **Call for network training** method.
  - `disaggregate_chunk`：**Call for network testing** method and **Return predictions**
  - `call_preprocessing`: **Pre-process** the training and testing data (**sliding window generation** (implementations are different with Seq2Seq, Seq2Point and Seq2Subseq) and **scale normalization** such as z-score)
  - `set_appliance_params`: As the name suggests

  Since the relevant code is redundant, so:

  if your network is Seq2Seq mapping, you can take use of dae_pytorch.py

  if your network is Seq2Point mapping, you can take use of bilstm_pytorch.py

  if your network is Seq2Subseq mapping, you can take use of sgn_pytorch.py

  In the future, we will write some sealed APIs to simplify and beatify the code. 

In [5]:
class Seq2SeqCNN(Disaggregator):    
    def __init__(self, params):
        self.MODEL_NAME = "Seq2SeqCNN"
        self.sequence_length = params.get('sequence_length',129)
        self.n_epochs = params.get('n_epochs', 10)
        self.batch_size = params.get('batch_size',512)
        self.appliance_params = params.get('appliance_params',{})
        self.mains_mean = params.get('mains_mean',None)
        self.mains_std = params.get('mains_std',None)
        self.models = OrderedDict()       
        
    def partial_fit(self, train_main, train_appliances, pretrain = False, do_preprocessing=True,pretrain_path = "./seq2seqcnn_pre_state_dict.pkl",**load_kwargs):         
        # If no appliance wise parameters are specified, then they are computed from the data
        if len(self.appliance_params) == 0:
            self.set_appliance_params(train_appliances)

        # Preprocess the data and bring it to a valid shape
        if do_preprocessing:
            print ("Doing Preprocessing")
            train_main,train_appliances = self.call_preprocessing(train_main,train_appliances,'train')

        train_main = pd.concat(train_main,axis = 0).values
        train_main = train_main.reshape((-1,self.sequence_length,1))

        new_train_appliances  = []
        for app_name, app_df in train_appliances:
            app_df = pd.concat(app_df,axis=0).values
            app_df = app_df.reshape((-1,self.sequence_length,1))
            new_train_appliances.append((app_name, app_df))
        train_appliances = new_train_appliances
        for appliance_name, power in train_appliances:
            if appliance_name not in self.models:
                print ("First model training for ",appliance_name)
                self.models[appliance_name] = seq2seqcnn_Pytorch(self.sequence_length)
                # Load pretrain dict or not
                if pretrain is True:
                    self.models[appliance_name].load_state_dict(torch.load("./"+appliance_name+"_seq2seqcnn_pre_state_dict.pt"))
  
            model = self.models[appliance_name]
            train(appliance_name, model, train_main, power, self.n_epochs, self.batch_size, pretrain, checkpoint_interval = 3)
            # Model test will be based on the best model
            self.models[appliance_name].load_state_dict(torch.load("./"+appliance_name+"_seq2seqcnn_best_state_dict.pt"))


    def disaggregate_chunk(self, test_main_list, do_preprocessing = True):
        # Disaggregate (test process)
        if do_preprocessing:
            test_main_list = self.call_preprocessing(test_main_list,submeters_lst = None,method='test')

        test_predictions = []
        for test_main in test_main_list:
            test_main = test_main.values.reshape((-1,self.sequence_length,1))
            disggregation_dict = {}

            for appliance in self.models:
                # Move the model to cpu, and then test it
                model = self.models[appliance].to('cpu')
                prediction = test(model, test_main)
                app_mean, app_std = self.appliance_params[appliance]['mean'], self.appliance_params[appliance]['std']
                prediction = self.denormalize_output(prediction,app_mean,app_std)
                valid_predictions = prediction.flatten()
                valid_predictions = np.where(valid_predictions > 0, valid_predictions, 0)
                series = pd.Series(valid_predictions)
                disggregation_dict[appliance] = series
            results = pd.DataFrame(disggregation_dict, dtype = 'float32')
            test_predictions.append(results)
        return test_predictions

    def call_preprocessing(self, mains_lst, submeters_lst, method):
        # Seq2Seq Version
        sequence_length  = self.sequence_length
        if method=='train':
            # Preprocess the main and appliance data, the parameter 'overlapping' will be set 'True'
            processed_mains = []
            for mains in mains_lst:
                self.mains_mean, self.mains_std = mains.values.mean(), mains.values.std()               
                mains = self.normalize_data(mains.values,sequence_length, mains.values.mean(), mains.values.std(),True)
                processed_mains.append(pd.DataFrame(mains))

            tuples_of_appliances = []
            for (appliance_name,app_df_list) in submeters_lst:
                app_mean = self.appliance_params[appliance_name]['mean']
                app_std = self.appliance_params[appliance_name]['std']
                processed_app_dfs = []
                for app_df in app_df_list:
                    data = self.normalize_data(app_df.values, sequence_length,app_mean,app_std,True)
                    processed_app_dfs.append(pd.DataFrame(data))                    
                tuples_of_appliances.append((appliance_name, processed_app_dfs))

            return processed_mains, tuples_of_appliances

        if method=='test':
            # Preprocess the main data only, the parameter 'overlapping' will be set 'False'
            processed_mains = []
            for mains in mains_lst:                
                mains = self.normalize_data(mains.values,sequence_length, mains.values.mean(), mains.values.std(),False)
                processed_mains.append(pd.DataFrame(mains))
            return processed_mains
        
    def normalize_data(self,data,sequence_length, mean, std, overlapping = False):
        # If you want to train the model,then overlapping = True will bring you a lot more training data; else overlapping = false to disaggregate the mains data
        n = sequence_length
        excess_entries =  sequence_length - (data.size % sequence_length)       
        lst = np.array([0] * excess_entries)
        arr = np.concatenate((data.flatten(), lst), axis = 0)   
        if overlapping:
            windowed_x = np.array([ arr[i:i+n] for i in range(len(arr)-n+1) ])
        else:
            windowed_x = arr.reshape((-1,sequence_length))
        # z-score normalization: y = (x - mean)/std
        windowed_x = windowed_x - mean
        return (windowed_x / std).reshape((-1,sequence_length))

    def denormalize_output(self,data,mean,std):
        # x = y * std + mean
        return mean + data * std
    
    def set_appliance_params(self,train_appliances):
        # Set appliance mean and std to normalize the label(appliance data)
        for (app_name, df_list) in train_appliances:
            l = np.array(pd.concat(df_list, axis=0))
            app_mean = np.mean(l)
            app_std = np.std(l)
            self.appliance_params.update({app_name:{'mean':app_mean,'std':app_std}})

- Finally, add your model to `\nilmtk\disaggregate\__init__.py` and enjoy it!

### How to write your own Evaluation Metrics

We will take `MCC` as an example,which has already been incorporated in the written framework

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, recall_score, precision_score, matthews_corrcoef
import numpy as np
on_threhold = {'fridge':50, 'kettle':2000, 'dish washer':10, 'washing machine':20, 'drill':0}

def MCC(app_name,app_gt, app_pred):
    threshold = on_threhold.get(app_name,10)
    gt_temp = np.array(app_gt)
    gt_temp = np.where(gt_temp < threshold, 0, 1)
    pred_temp = np.array(app_pred)
    pred_temp = np.where(pred_temp < threshold,0, 1)

    return matthews_corrcoef(gt_temp, pred_temp)

From above code we can know, there are certain **on-power threholds** for cretain appliances, you can modify them as you need.And then you can write your own metrics by the means of **scikit-learn** which takes (app_name,app_gt, app_pred) as parameters.